In [15]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.ndimage import gaussian_filter1d

In [16]:
np.random.seed(5)
np.set_printoptions(threshold=sys.maxsize)

In [17]:
x = [1,2,3,4,5]
y = slice(0,2)
x[y]

[1, 2]

In [26]:
def calc_reps(T, start_time, end_time, inp_num, rec_num, inh_num, inh_duration, inp_delay_range, rec_delay_range,V_th):
    
    
    inp_event_max = inp_delay_range[1]-inp_delay_range[0]

    inp_delays = np.random.randint(*inp_delay_range,(inp_num ,rec_num))
    rec_delays = np.random.randint(*rec_delay_range,(rec_num+inp_num+inh_num,rec_num+inp_num+inh_num))
    inp_delays.shape,rec_delays.shape
    
    all_input_events = np.zeros((inp_event_max**inp_num, T, inp_num))
    all_rec_events = np.zeros((inp_event_max**inp_num, T,rec_num))
    all_events = np.zeros((inp_event_max**inp_num,T,inp_num+rec_num+inh_num))
    

    idc = 0
    for i in range(inp_event_max):
        for j in range(inp_event_max):
            all_input_events[idc,j+start_time,slice(0,inp_num)] = 1
            idc +=1
            
    
# all_input_events[-1,:,:]
    idc=0
    for inp_events in all_input_events:
        rec_events = np.zeros((T,rec_num+inh_num))
    #     inp = np.random.randint(0,10,(inp_num))
    #     inp_events = np.zeros((T,inp_num))
    #     for i in range(inp_num):
    #         inp_events[start_time+inp[i],i] =1
    #     inp_events.shape

        for t in range(1+start_time,end_time):
            for j in range(rec_num):
                I = 0

                for i in range(inp_num):
                    I += inp_events[t-inp_delays[i,j],i]
                for i in range(rec_num+inh_num):
                    if i!=j:
                        I += rec_events[t-rec_delays[i,j],i]

                if I>V_th:
                    rec_events[t,j] =1
                else:
                    rec_events[t,j] =0

            for j in range(rec_num,rec_num+inh_num):
                I = 0

                for i in range(rec_num):
                    if i!=j:
                        I += rec_events[t-rec_delays[i,j],i]


                if I>1:
                    rec_events[t,j] =-2
                else:
                    rec_events[t,j] =0

        events = np.zeros((T,inp_num+rec_num+inh_num))
        events[:,:inp_num] = inp_events
        events[:,inp_num:inp_num+rec_num+inh_num] = rec_events
        all_events[idc]=events
    #     all_rec_events[]
        idc+=1
    
    return all_events
    

# all_rec_events, evoked_spikes, temporal_depth,  unique_signatures = calc_reps(**params)

In [31]:

T = 100 # miliseconds of simulation time
start_time=10
end_time=90
inp_num = 2
rec_num = 4
inh_num = 2
nrns = np.array((inp_num*[0]+rec_num*[1]+inh_num*[2]))

inh_duration = 2
inp_delay_range=(1,5)
rec_delay_range=(1,5)
inp_event_max=4
V_th=1

default_params = {
    'T' : T, # miliseconds of simulation time
    'start_time':start_time,
    'end_time':end_time,
    'inp_num' : inp_num,
    'rec_num' : rec_num,
    'inh_num' : inh_num,
    'inh_duration' : inh_duration,
    'inp_delay_range':inp_delay_range,
    'rec_delay_range':rec_delay_range,
    'V_th':V_th
}

colors = np.array(['#1f77b4', '#1f77b4', '#1f77b4', '#d62728', '#d62728', '#d62728', '#d62728', '#d62728', '#d62728', '#17becf'])
colors = np.array(inp_num*['#1f77b4'] + rec_num*['#d62728'] + inh_num*['#17becf'])

In [32]:
trials = 1

all_events = np.zeros((trials, inp_event_max**inp_num,T,inp_num+rec_num+inh_num))

for trial in tqdm(range(trials)):
    all_events[trial] = calc_reps(**default_params)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.74it/s]


# statistics!

In [40]:
scan_dict = {
    'inp_num':[2,3],
    'rec_num':[2,3,4,5,6,9,13,20],
    'inh_num':[0,1,2,3,4,8,12],
    'inh_duration':[1,2,3,4,5],
    'inp_delay_range':[(1,5),(1,6),(1,7),(1,8),(1,10),(1,15),(1,20)],
    'rec_delay_range':[(1,5),(1,6),(1,7),(1,10)],
    'V_th':[1,2,3,4,5]
}

In [41]:
len(scan_dict['inp_num'])

2

In [42]:
trials = 1

events_dict = {}
for key_idc, key in enumerate(scan_dict):
    
    events_dict[key] = {}
    
    for val_idc, val in enumerate(scan_dict[key]):
        
        
        
        params = default_params.copy()
        params[key]=val
        

        all_events = np.zeros((trials, inp_event_max**inp_num,T,inp_num+rec_num+inh_num))

        for trial in tqdm(range(trials)):
            all_events[trial] = calc_reps(**params)
            
        events_dict[key][val] = all_events


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


ValueError: could not broadcast input array from shape (64,100,9) into shape (16,100,8)